In [5]:
import os
import platform
from IPython.display import clear_output
print(platform.platform())

def resolve_dir(Dir):
    if not os.path.exists(Dir):
        os.mkdir(Dir)

def reset_path(Dir):
    if not os.path.exists(Dir):
        os.mkdir(Dir)
    else:
        os.system('rm -f {}/*'.format( Dir))

macOS-13.2.1-arm64-arm-64bit


In [6]:
import tensorflow as tf
tf.random.set_seed(73)
TPU_INIT = False

if TPU_INIT:
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime!')
else:
    !nvidia-smi
;    
print("Tensorflow version " + tf.__version__)

zsh:1: command not found: nvidia-smi
Tensorflow version 2.13.0-rc2


In [7]:
import cv2
import os
import imageio
import imgaug.augmenters as iaa
import imgaug as ia

IMG_SIZE = 128
ColorChannels = 3

def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)
    
    import math
    rate = math.floor(vidcap.get(3))
    count = 0
    
    ImageFrames = []
    while vidcap.isOpened():
        ID = vidcap.get(1)
        success, image = vidcap.read()
        
        if success:
            if (ID % 7 == 0):
                flip = iaa.Fliplr(1.0)
                zoom = iaa.Affine(scale=1.3)
                random_brightness = iaa.Multiply((1, 1.3))
                rotate = iaa.Affine(rotate=(-25, 25))
                
                image_aug = flip(image = image)
                image_aug = random_brightness(image = image_aug)
                image_aug = zoom(image = image_aug)
                image_aug = rotate(image = image_aug)
                
                rgb_img = cv2.cvtColor(image_aug, cv2.COLOR_BGR2RGB)
                resized = cv2.resize(rgb_img, (IMG_SIZE, IMG_SIZE))
                ImageFrames.append(resized)
            count+=1
        else:
            break
    
    vidcap.release()
    
    return ImageFrames

In [8]:
print('we have \n{} Violence videos \n{} NonViolence videos'.format(
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence')), 
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence'))))

we have 
1000 Violence videos 
1000 NonViolence videos


In [9]:
%%time
#tqdm is used here to visulaize the progress of dataset creation
from tqdm import tqdm

VideoDataDir ='/Users/faizahkureshi/Desktop/GAIP Project/violencedataset'
print('we have \n{} Violence videos \n{} NonViolence videos'.format(
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence')), 
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence'))))

X_original = []
y_original = []
CLASSES = ["NonViolence", "Violence"]

for category in os.listdir(VideoDataDir):
    category_path = os.path.join(VideoDataDir, category)
    if os.path.isdir(category_path):
        class_num = CLASSES.index(category)
        try:
            video_list = [video for video in os.listdir(category_path)[:350] if video != '.DS_Store']
            for i, video in enumerate(tqdm(video_list)):
                video_path = os.path.join(category_path, video)
                frames = video_to_frames(video_path)
                for j, frame in enumerate(frames):
                    X_original.append(frame)
                    y_original.append(class_num)
        except OSError as e:
            print(f"Error processing directory: {category_path}. Skipping...")
            continue



we have 
1000 Violence videos 
1000 NonViolence videos


100%|██████████| 350/350 [00:46<00:00,  7.54it/s]

CPU times: user 2min 45s, sys: 5.22 s, total: 2min 50s
Wall time: 1min 5s


In [11]:
import numpy as np
X_original = np.array(X_original).reshape(-1 , IMG_SIZE * IMG_SIZE * 3)
y_original = np.array(y_original)
len(X_original)

13487

In [12]:
len(y_original)

13487

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

stratified_sample = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=73)

for train_index, test_index in stratified_sample.split(X_original, y_original):
    X_train, X_test = X_original[train_index], X_original[test_index]
    y_train, y_test = y_original[train_index], y_original[test_index]

X_train_nn = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test_nn = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255

In [14]:
len(X_test_nn)

4047

In [15]:
len(X_train_nn)

9440

In [16]:
len(y_test)

4047

In [17]:
len(y_train)

9440

In [18]:
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")

In [19]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# Set the input shape
input_shape = (128, 128, 3)


# Create the base ResNet50 model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

# Freeze the base model's layers
base_model.trainable = False

# Create the custom head for classification
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256, activation='selu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='selu', kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)




In [20]:
# Train the model
history = model.fit(X_train_nn, y_train, batch_size=32, epochs=50, validation_data=(X_test_nn, y_test), callbacks=[early_stopping])


Epoch 1/50
295/295 [==============================] - 153s 515ms/step - loss: 1.8777 - accuracy: 0.7002 - val_loss: 1.2113 - val_accuracy: 0.6943
Epoch 2/50
295/295 [==============================] - 147s 500ms/step - loss: 0.9654 - accuracy: 0.7514 - val_loss: 0.8161 - val_accuracy: 0.7759
Epoch 3/50
295/295 [==============================] - 149s 504ms/step - loss: 0.7620 - accuracy: 0.7606 - val_loss: 0.7420 - val_accuracy: 0.7361
Epoch 4/50
295/295 [==============================] - 149s 507ms/step - loss: 0.6955 - accuracy: 0.7673 - val_loss: 0.6893 - val_accuracy: 0.7576
Epoch 5/50
295/295 [==============================] - 154s 523ms/step - loss: 0.6528 - accuracy: 0.7675 - val_loss: 0.6419 - val_accuracy: 0.7700
Epoch 6/50
295/295 [==============================] - 141s 477ms/step - loss: 0.6225 - accuracy: 0.7788 - val_loss: 0.6834 - val_accuracy: 0.7257
Epoch 7/50
295/295 [==============================] - 140s 475ms/step - loss: 0.6024 - accuracy: 0.7854 - val_loss: 0.6582 -

In [4]:
          
# Evaluate the model
loss, accuracy = model.evaluate(X_test_nn, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

NameError: name 'X_test_nn' is not defined

In [3]:
from keras.models import load_model
model2 = load_model('/Users/faizahkureshi/Desktop/GAIP Project/modelnew.h5')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Assuming you have the predicted probabilities and true labels for both models
# pred_prob_model1: predicted probabilities for model 1
# true_labels: true labels

# Calculate the false positive rate (fpr) and true positive rate (tpr) for model 1
fpr_model1, tpr_model1, _ = roc_curve(true_labels, pred_prob_model1)

# Calculate the false positive rate (fpr) and true positive rate (tpr) for model 2
fpr_model2, tpr_model2, _ = roc_curve(true_labels, pred_prob_model2)

# Calculate the area under the ROC curve (AUC) for both models
auc_model1 = roc_auc_score(true_labels, pred_prob_model1)
auc_model2 = roc_auc_score(true_labels, pred_prob_model2)

# Plot the ROC curve
plt.plot(fpr_model1, tpr_model1, label='Model 1 (AUC = {:.2f})'.format(auc_model1))
plt.plot(fpr_model2, tpr_model2, label='Model 2 (AUC = {:.2f})'.format(auc_model2))

# Set labels and title for the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

# Show the plot
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming you have the true labels and predicted labels stored in variables
true_labels =y_test
predicted_labels = predictions

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)